# Planning to Joint Configurations 🦾

In this notebook we show how you can setup a planner to find collision-free joint paths for a robot between any two joints configurations.

## 1. Setting up the scene & collision checker 🏗️️

For more details about this, see the [`airo-drake`](https://github.com/airo-ugent/airo-drake) notebooks.


In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import SingleArmScene, add_floor, add_manipulator, add_meshcat, finish_build
from pydrake.planning import SceneGraphCollisionChecker

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur3e", "robotiq_2f_85", static_gripper=True)
add_floor(robot_diagram_builder)
robot_diagram, context = finish_build(robot_diagram_builder)

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)


collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[scene.arm_index, scene.gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

collision_checker.CheckConfigCollisionFree(np.zeros(6))

## 2. Creating the motion planner 🧭

In [ ]:
from airo_planner import SingleArmOmplPlanner


planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree)

## 3. Planning to joint configurations 🦾

In [ ]:
start_joints = np.deg2rad([0, -90, 90, -90, -90, 0])
goal_joints = np.deg2rad([0, -90, 90, 180, -110, 0])

path = planner.plan_to_joint_configuration(start_joints, goal_joints)

In [ ]:
from airo_drake import calculate_joint_path_length

calculate_joint_path_length(path)

In [ ]:
from airo_drake import animate_joint_configurations

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, path)

In [ ]:
from airo_drake import time_parametrize_toppra, animate_joint_trajectory

trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_joint_trajectory(scene.meshcat, scene.robot_diagram, scene.arm_index, trajectory)